# The Simplex algotithm

A simple implementation for the primal formulation of the simplex algorithm.

## Formulation of a simple problem

In [53]:
import numpy as np
import attr
from fractions import Fraction

In [54]:
_A = np.array(
    [
        [1, 3, 1, 1, 0, 0, 0],
        [-1, 0, 3, 0, 1, 0, 0],
        [2, -1, 2, 0, 0, 1, 0],
        [2, 3, -1, 0, 0, 0, 1]
    ],
    dtype=np.int64
)

_A

array([[ 1,  3,  1,  1,  0,  0,  0],
       [-1,  0,  3,  0,  1,  0,  0],
       [ 2, -1,  2,  0,  0,  1,  0],
       [ 2,  3, -1,  0,  0,  0,  1]])

In [55]:
_p = np.array([3, 2, 4, 2], dtype=np.int64)
_p

array([3, 2, 4, 2])

In [56]:
_B = np.hstack([_A, _p[..., np.newaxis]])
_B

array([[ 1,  3,  1,  1,  0,  0,  0,  3],
       [-1,  0,  3,  0,  1,  0,  0,  2],
       [ 2, -1,  2,  0,  0,  1,  0,  4],
       [ 2,  3, -1,  0,  0,  0,  1,  2]])

In [57]:
_z = np.array([5, 5, 3, 0, 0, 0, 0] + [0], dtype=np.int64)
_z

array([5, 5, 3, 0, 0, 0, 0, 0])

In [58]:
from mcf_simplex_analyzer.fractionarray import FractionArray

In [59]:
A = FractionArray(_A, np.ones_like(_A))
A

FractionArray(numerator=[[ 1  3  1  1  0  0  0]
 [-1  0  3  0  1  0  0]
 [ 2 -1  2  0  0  1  0]
 [ 2  3 -1  0  0  0  1]], denominator=[[1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]])

In [60]:
A[..., 1]

FractionArray(numerator=[ 3  0 -1  3], denominator=[1 1 1 1])

In [61]:
p = FractionArray(_p, np.ones_like(_p))
p

FractionArray(numerator=[3 2 4 2], denominator=[1 1 1 1])

In [62]:
B = FractionArray(_B, np.ones_like(_B))
B

FractionArray(numerator=[[ 1  3  1  1  0  0  0  3]
 [-1  0  3  0  1  0  0  2]
 [ 2 -1  2  0  0  1  0  4]
 [ 2  3 -1  0  0  0  1  2]], denominator=[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]])

In [63]:
base_ind = np.array([3, 4, 5, 6, -1])
base = np.zeros_like(_B[0], dtype=bool)
base[base_ind] = True
base

array([False, False, False,  True,  True,  True,  True,  True])

In [64]:
# TODO: How to determine this?
row_to_var_index = np.array([3, 4, 5, 6])
var_index_to_row = np.array([0, 0, 0, 0, 1, 2, 3])

In [65]:
z = FractionArray(_z, np.ones_like(_z))
z

FractionArray(numerator=[5 5 3 0 0 0 0 0], denominator=[1 1 1 1 1 1 1 1])

## Entering variable (Dantzig)

In [66]:
def dantzig_entering(z, base):
    positive = np.where(~base)[0]
    entering = positive[z[positive].argmax()]
    
    return entering

entering = dantzig_entering(z, base)
entering

0

In [67]:
# Possible speedup: Convert to floats
rng = np.random.default_rng()
x = rng.uniform(size=1000)
m = x.max()
eps = 0.001
np.where((m - eps <= x) & (x <= m + eps))[0]

array([210, 856])

## Leaving variable (Dantzig)

In [68]:
def dantzig_leaving(T, entering):   
    positive = np.where(T[..., entering].numerator >= 0)[0]
    
    bounds = T[..., -1][positive] / T[..., entering][positive]
    print("Bounds:", bounds)
    valid = np.where(bounds.numerator > 0)[0]
    leaving_row = positive[valid[bounds[valid].argmin()]]
    
    return leaving_row

In [69]:
leaving_row = dantzig_leaving(B, entering)
leaving_row

Bounds: FractionArray(numerator=[3 2 1], denominator=[1 1 1])


3

In [70]:
leaving = row_to_var_index[leaving_row]
leaving

6

## Update

In [71]:
def determine_update_row(B, entering, leaving_row):
    if B[leaving_row, entering] == 0:
        return FractionArray.from_array(np.ones_like(B[leaving_row].numerator))
    

    return B[leaving_row] / B[leaving_row, entering]

In [72]:
update_row = determine_update_row(B, entering, leaving_row)
update_row

FractionArray(numerator=[ 1  3 -1  0  0  0  1  1], denominator=[1 2 2 1 1 1 2 1])

In [73]:
B

FractionArray(numerator=[[ 1  3  1  1  0  0  0  3]
 [-1  0  3  0  1  0  0  2]
 [ 2 -1  2  0  0  1  0  4]
 [ 2  3 -1  0  0  0  1  2]], denominator=[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]])

In [74]:
def update_table(B, update_row, entering, leaving_row):
    B[leaving_row] = update_row
    
    for row in range(B.numerator.shape[0]):
        if row == leaving_row:
            continue
            
        B[row] -= B[row, entering] * update_row

In [75]:
update_table(B, update_row, entering, leaving_row)
B

FractionArray(numerator=[[ 0  3  3  1  0  0 -1  2]
 [ 0  3  5  0  1  0  1  3]
 [ 0 -4  3  0  0  1 -1  2]
 [ 1  3 -1  0  0  0  1  1]], denominator=[[1 2 2 1 1 1 2 1]
 [1 2 2 1 1 1 2 1]
 [1 1 1 1 1 1 1 1]
 [1 2 2 1 1 1 2 1]])

In [76]:
def update_objective(z, update_row, entering, leaving_row):
    update = update_row * z[entering]
    update[:-1] *= -1
    
    z += update

In [77]:
update_objective(z, update_row, entering, leaving_row)
z

FractionArray(numerator=[ 0 -5 11  0  0  0 -5  5], denominator=[1 2 2 1 1 1 2 1])

In [78]:
def update_base(base, entering, leaving_row):
    leaving = row_to_var_index[leaving_row]
    row_to_var_index[leaving_row] = entering
    
    var_index_to_row[entering] = var_index_to_row[leaving]

    base[entering] = True
    base[leaving] = False
    
    return base
    
update_base(base, entering, leaving_row)
base

array([ True, False, False,  True,  True,  True, False,  True])

# Second iteration

In [79]:
entering2 = dantzig_entering(z, base)
entering2

2

In [80]:
leaving_row2 = dantzig_leaving(B, entering2)

Bounds: FractionArray(numerator=[4 6 2], denominator=[3 5 3])


In [81]:
leaving2 = row_to_var_index[leaving_row2]
leaving2

5

In [82]:
update_row = determine_update_row(B, entering2, leaving_row2)
update_row

FractionArray(numerator=[ 0 -4  1  0  0  1 -1  2], denominator=[1 3 1 1 1 3 3 3])

In [83]:
update_table(B, update_row, entering2, leaving_row2)
B

FractionArray(numerator=[[ 0  7  0  1  0 -1  0  1]
 [ 0 29  0  0  1 -5  4  4]
 [ 0 -4  1  0  0  1 -1  2]
 [ 1  5  0  0  0  1  1  4]], denominator=[[1 2 1 1 1 2 1 1]
 [1 6 1 1 1 6 3 3]
 [1 3 1 1 1 3 3 3]
 [1 6 1 1 1 6 3 3]])

In [84]:
type(Fraction(3, 2))

fractions.Fraction

In [85]:
update_objective(z, update_row, entering2, leaving_row2)
z

FractionArray(numerator=[  0  29   0   0   0 -11  -2  26], denominator=[1 6 1 1 1 6 3 3])

In [86]:
update_base(base, entering2, leaving_row2)
base

array([ True, False,  True,  True,  True, False, False,  True])

In [87]:
print("Objective function:", z[-1])
for var in row_to_var_index:
    print(f"x_{var}:", B[var_index_to_row[var], -1])

Objective function: 26/3
x_3: 1
x_4: 4/3
x_2: 2/3
x_0: 4/3


## Third iteration

In [88]:
entering2 = dantzig_entering(z, base)

leaving_row2 = dantzig_leaving(B, entering2)

leaving2 = row_to_var_index[leaving_row2]

update_row = determine_update_row(B, entering2, leaving_row2)

update_table(B, update_row, entering2, leaving_row2)
update_objective(z, update_row, entering2, leaving_row2)
update_base(base, entering2, leaving_row2)

Bounds: FractionArray(numerator=[2 8 8], denominator=[ 7 29  5])


array([ True,  True,  True,  True, False, False, False,  True])

In [89]:
print("Objective function:", z[-1])
for var in row_to_var_index:
    print(f"x_{var}:", B[var_index_to_row[var], -1])

Objective function: 10
x_3: 1/29
x_1: 8/29
x_2: 30/29
x_0: 32/29


## Simplex class

In [90]:
from enum import Enum

class LPFormType(Enum):
    Canonical = 0
    Standard = 1

In [91]:
@attr.s
class LPFormulation:
    type : LPFormType = attr.ib()
        
    table: FractionArray = attr.ib()
    rhs: FractionArray = attr.ib()
    objective: FractionArray = attr.ib()
        
    meta: dict = attr.ib(kw_only=True, factory=dict)

In [92]:
attr.fields(LPFormulation)

(Attribute(name='type', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=<enum 'LPFormType'>, converter=None, kw_only=False, inherited=False, on_setattr=None),
 Attribute(name='table', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=<class 'mcf_simplex_analyzer.fractionarray.FractionArray'>, converter=None, kw_only=False, inherited=False, on_setattr=None),
 Attribute(name='rhs', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=<class 'mcf_simplex_analyzer.fractionarray.FractionArray'>, converter=None, kw_only=False, inherited=False, on_setattr=None),
 Attribute(name='objective', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, 

In [93]:
_C = np.array(
    [
        [1, 3, 1 ],
        [-1, 0, 3 ],
        [2, -1, 2 ],
        [2, 3, -1 ]
    ],
    dtype=np.int64
)

C = FractionArray.from_array(_C)
C

FractionArray(numerator=[[ 1  3  1]
 [-1  0  3]
 [ 2 -1  2]
 [ 2  3 -1]], denominator=[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]])

In [94]:
_p = np.array([3, 2, 4, 2], dtype=np.int64)
_p
p = FractionArray.from_array(_p)
p

FractionArray(numerator=[3 2 4 2], denominator=[1 1 1 1])

In [95]:
_z = np.array([5, 5, 3], dtype=np.int64)
z = FractionArray.from_array(_z)
z

FractionArray(numerator=[5 5 3], denominator=[1 1 1])

In [96]:
form = LPFormulation(LPFormType.Canonical, C, p, z)
form

LPFormulation(type=<LPFormType.Canonical: 0>, table=FractionArray(numerator=[[ 1  3  1]
 [-1  0  3]
 [ 2 -1  2]
 [ 2  3 -1]], denominator=[[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]]), rhs=FractionArray(numerator=[3 2 4 2], denominator=[1 1 1 1]), objective=FractionArray(numerator=[5 5 3], denominator=[1 1 1]), meta={})

In [97]:
@attr.s(kw_only=True)
class Simplex:
    table = attr.ib()
    objective = attr.ib()
    
    _base = attr.ib()
    _row_to_var_index = attr.ib()
    _var_index_to_row = attr.ib()
    
    @classmethod
    def _from_canonical(cls, form: LPFormulation):
        m, n = form.table.shape
        if np.all(form.rhs >= 0):
            slack = np.eye(m, dtype=form.table.numerator.dtype)
            new_table = FractionArray(
                np.hstack([form.table.numerator, slack, form.rhs.numerator[..., np.newaxis]]),
                np.hstack([form.table.denominator, np.ones_like(slack), form.rhs.denominator[..., np.newaxis]])
            )

            new_objective = FractionArray(
                np.hstack([form.objective.numerator, np.zeros(m, dtype=form.objective.numerator.dtype), [0]]),
                np.hstack([form.objective.denominator, np.ones(m, dtype=form.objective.denominator.dtype), [1]])
            )
            
            base = np.hstack([np.zeros(n, dtype=bool), np.ones(m, dtype=bool)])
            
            row_to_var_index = np.arange(m, dtype=int) + n
            var_index_to_row = np.hstack([np.zeros(n, dtype=int), np.arange(m, dtype=int)])
            
            return cls(table=new_table,
                       objective=new_objective,
                       base=base,
                       row_to_var_index=row_to_var_index,
                       var_index_to_row=var_index_to_row)
        else:
            raise NotImplementedError("Two phase simplex not implemented.")
        
    @classmethod
    def from_formulation(cls, formulation: LPFormulation):
        if formulation.type == LPFormType.Canonical:
            return cls._from_canonical(formulation)
        else:
            raise NotImplemented
       
    def _get_update_row(self, entering, leaving_row):
        return self.table[leaving_row] / self.table[leaving_row, entering]
    
    def _update_objective(self, update_row, entering, leaving_row):
        update = update_row * self.objective[entering]
        update[:-1] *= -1

        self.objective += update
        
    def _update_base(self, entering, leaving_row):
        leaving = self._row_to_var_index[leaving_row]
        self._row_to_var_index[leaving_row] = entering

        self._var_index_to_row[entering] = self._var_index_to_row[leaving]

        self._base[entering] = True
    
    def _update_table(self, update_row, entering, leaving_row):
        self.table[leaving_row] = update_row

        for row in range(self.table.shape[0]):
            if row == leaving_row:
                continue

            self.table[row] -= self.table[row, entering] * update_row
        
    def _is_end(self):
        return np.all(self.objective[:-1][~self._base] < 0)
        
        
    def solve(self):
        while True:
            print(self.objective)
            print("Objective function:", self.objective)
            for var in self._row_to_var_index:
                print(f"x_{var}:", self.table[self._var_index_to_row[var], -1])
            
            # Check end
            if self._is_end():
                print("End")
                break
            
            # Determine entering
            entering = dantzig_entering(self.objective[:-1], self._base)
            print("Entering:", entering)
            
            # Check unbounded
            if np.all(self.table[..., entering] < 0):
                print("Unbounded")
                break
            
            # Determine leaving
            leaving_row = dantzig_leaving(self.table, entering)
            print("Leaving row:", leaving_row)
            
            # Update
            update_row = self._get_update_row(entering, leaving_row)

            self._update_table(update_row, entering, leaving_row)
            self._update_objective(update_row, entering, leaving_row)
            self._update_base(entering, leaving_row)

attr.fields(Simplex)

(Attribute(name='table', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=True, inherited=False, on_setattr=None),
 Attribute(name='objective', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=True, inherited=False, on_setattr=None),
 Attribute(name='_base', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=True, inherited=False, on_setattr=None),
 Attribute(name='_row_to_var_index', default=NOTHING, validator=None, repr=True, eq=True, eq_key=None, order=True, order_key=None, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=True, inherited=False, on_setattr=None),
 Attribu

In [98]:
s = Simplex.from_formulation(form)
s

Simplex(table=FractionArray(numerator=[[ 1  3  1  1  0  0  0  3]
 [-1  0  3  0  1  0  0  2]
 [ 2 -1  2  0  0  1  0  4]
 [ 2  3 -1  0  0  0  1  2]], denominator=[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]), objective=FractionArray(numerator=[5 5 3 0 0 0 0 0], denominator=[1 1 1 1 1 1 1 1]), _base=array([False, False, False,  True,  True,  True,  True]), _row_to_var_index=array([3, 4, 5, 6]), _var_index_to_row=array([0, 0, 0, 0, 1, 2, 3]))

In [99]:
s.table[..., :-1]

FractionArray(numerator=[[ 1  3  1  1  0  0  0]
 [-1  0  3  0  1  0  0]
 [ 2 -1  2  0  0  1  0]
 [ 2  3 -1  0  0  0  1]], denominator=[[1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]])

In [100]:
s.solve()

FractionArray(numerator=[5 5 3 0 0 0 0 0], denominator=[1 1 1 1 1 1 1 1])
Objective function: FractionArray(numerator=[5 5 3 0 0 0 0 0], denominator=[1 1 1 1 1 1 1 1])
x_3: 3
x_4: 2
x_5: 4
x_6: 2
Entering: 0
Bounds: FractionArray(numerator=[3 2 1], denominator=[1 1 1])
Leaving row: 3
FractionArray(numerator=[ 0 -5 11  0  0  0 -5  5], denominator=[1 2 2 1 1 1 2 1])
Objective function: FractionArray(numerator=[ 0 -5 11  0  0  0 -5  5], denominator=[1 2 2 1 1 1 2 1])
x_3: 2
x_4: 3
x_5: 2
x_0: 1
Entering: 2
Bounds: FractionArray(numerator=[4 6 2], denominator=[3 5 3])
Leaving row: 2
FractionArray(numerator=[  0  29   0   0   0 -11  -2  26], denominator=[1 6 1 1 1 6 3 3])
Objective function: FractionArray(numerator=[  0  29   0   0   0 -11  -2  26], denominator=[1 6 1 1 1 6 3 3])
x_3: 1
x_4: 4/3
x_2: 2/3
x_0: 4/3
Entering: 1
Bounds: FractionArray(numerator=[2 8 8], denominator=[ 7 29  5])
Leaving row: 1
FractionArray(numerator=[ 0  0  0  0 -1 -1 -2 10], denominator=[1 1 1 1 1 1 1 1])
Object

In [101]:
problem2 = LPFormulation(LPFormType.Canonical,
              table=FractionArray.from_array([[2,3,1], [4,1,2], [3, 4, 2]]),
              rhs=FractionArray.from_array([5, 11, 8]),
              objective=FractionArray.from_array([5,4,3])
             )
problem2

LPFormulation(type=<LPFormType.Canonical: 0>, table=FractionArray(numerator=[[2 3 1]
 [4 1 2]
 [3 4 2]], denominator=[[1 1 1]
 [1 1 1]
 [1 1 1]]), rhs=FractionArray(numerator=[ 5 11  8], denominator=[1 1 1]), objective=FractionArray(numerator=[5 4 3], denominator=[1 1 1]), meta={})

In [102]:
Simplex.from_formulation(problem2).solve()

FractionArray(numerator=[5 4 3 0 0 0 0], denominator=[1 1 1 1 1 1 1])
Objective function: FractionArray(numerator=[5 4 3 0 0 0 0], denominator=[1 1 1 1 1 1 1])
x_3: 5
x_4: 11
x_5: 8
Entering: 0
Bounds: FractionArray(numerator=[ 5 11  8], denominator=[2 4 3])
Leaving row: 0
FractionArray(numerator=[ 0 -7  1 -5  0  0 25], denominator=[1 2 2 2 1 1 2])
Objective function: FractionArray(numerator=[ 0 -7  1 -5  0  0 25], denominator=[1 2 2 2 1 1 2])
x_0: 5/2
x_4: 1
x_5: 1/2
Entering: 2
Bounds: FractionArray(numerator=[5 1 1], denominator=[1 0 1])
Leaving row: 0
FractionArray(numerator=[-1 -5  0 -3  0  0 15], denominator=[1 1 1 1 1 1 1])
Objective function: FractionArray(numerator=[-1 -5  0 -3  0  0 15], denominator=[1 1 1 1 1 1 1])
x_2: 5
x_4: 1
x_5: -2
End


/home/davidkubek/code/mcf-simplex-analyzer/src/mcf_simplex_analyzer/fractionarray.py:118: RuntimeWarning: divide by zero encountered in floor_divide
  return np.argmin((lcm // self.denominator) * self.numerator)
